In [19]:
# ORIGINAL CODE : https://machinelearningmastery.com/weighted-average-ensemble-with-python/
# By Jason Brownlee - MAY 2021

# Modifed by MAHAYASA ADIPUTRA - OCT 2022
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as mt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
le = LabelEncoder()
start_time = time.time()
# get the dataset
# Importing the dataset
dataset=pd.read_csv('https://snow.co.id/customerchurn/data/news_churn.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

#transform data into numeric value
dataset['HHIncome']=le.fit_transform(dataset['HHIncome'])
dataset['HomeOwnership']=le.fit_transform(dataset['HomeOwnership'])
dataset['Ethnicity']=le.fit_transform(dataset['Ethnicity'])
dataset['dummyforChildren']=le.fit_transform(dataset['dummyforChildren'])
dataset['Agerange']=le.fit_transform(dataset['Agerange'])
dataset['Language']=le.fit_transform(dataset['Language'])
dataset['State']=le.fit_transform(dataset['State'])
dataset['City']=le.fit_transform(dataset['City'])
dataset['County']=le.fit_transform(dataset['County'])
dataset['weeklyfee']=le.fit_transform(dataset['weeklyfee'])
dataset['Deliveryperiod']=le.fit_transform(dataset['Deliveryperiod'])
dataset['NielsenPrizm']=le.fit_transform(dataset['NielsenPrizm'])
dataset['rewardprogram']=le.fit_transform(dataset['rewardprogram'])
dataset['SourceChannel']=le.fit_transform(dataset['SourceChannel'])
#filling missing value with mean
#dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['Subscriber','SubscriptionID','Address','ZipCode'],axis=1)
y=dataset["Subscriber"]
y = le.fit_transform(y)
#imblearn SMOTE
#smote = SMOTE(random_state=42)
#X, y = smote.fit_resample(X, y)
#smote_enn = SMOTEENN()
smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# get a list of base models
def get_models():
	models = list()
	models.append(('xb', XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)))
	models.append(('rf', RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)))
#	models.append(('ab', AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')))
	return models
 
# evaluate each base model
def evaluate_models(models, X,y):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# evaluate the model
		acc=cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
		# store the performance
		scores.append(mean(acc))
		# report model performance
	return scores



# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X,y)
print(scores)
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
score=cross_val_score(ensemble, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

[0.9366926425629545, 0.9535072362843762]
F1 score: 0.886
STD F1 Score: 0.004


In [20]:
X

,HHIncome,HomeOwnership,Ethnicity,dummyforChildren,YearOfResidence,Agerange,Language,State,City,County,weeklyfee,Deliveryperiod,NielsenPrizm,rewardprogram,SourceChannel
0,1,1,23,0,1,1,12,0,32,0,11,1,1,0,6
1,8,0,33,1,7,5,7,0,21,1,2,20,5,0,37
2,12,0,17,0,23,7,7,0,28,1,4,20,5,1,27
3,3,0,33,0,23,8,7,0,42,1,12,23,4,0,9
4,16,0,55,1,10,5,7,0,25,1,1,20,2,0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23114,12,0,17,0,26,6,7,0,41,1,2,20,2,0,45
23115,9,0,55,0,35,8,7,0,6,1,5,1,1,29,14
23116,7,0,23,0,13,9,7,0,16,0,6,1,1,5,46
23117,15,0,33,0,19,7,7,0,38,1,8,1,5,0,44


In [21]:
rc=cross_val_score(ensemble, X, y, scoring='recall', cv=cv, n_jobs=-1)
rc=mean(rc)
rc

0.8930729834242352

In [22]:
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(ensemble, X, y, cv=5, scoring=scorer)
sp=mean(sp)
sp

0.8434903088167582

In [ ]:
auc=cross_val_score(ensemble, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
auc=mean(auc)
auc

0.9527098555452518

In [ ]:
import math

gmean=rc*sp
sqrtg = math.sqrt(gmean)
sqrtg

0.8679276505472169